In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import joblib
import os

In [5]:
cd ..

/Users/yunxizzy/code/theresa-sporn/WorkforceSentimentMonitoring


In [6]:
ls

MANIFEST.in                            lexicon/
Makefile                               notebooks/
Procfile                               raw_data/
README.md                              requirements.txt
WorkforceSentimentMonitoring/          scripts/
WorkforceSentimentMonitoring.egg-info/ setup.py
app.py                                 setup.sh
build/                                 test_data/
dist/                                  tests/
joblib_files/


# Redo mnb feature extraction

In [13]:
# load train test data
from WorkforceSentimentMonitoring.data import get_prepaired_data

In [14]:
X_train, X_test, y_train, y_test = get_prepaired_data()

Reading data...
Merging data into a single DataFrame...
Dropping initial text columns...
Identifying entries in other languages...



Drop 430 entries? [y] / n
y
Dropping 430 entries...
Splitting train and test...
Encoding targets...
Done!


In [17]:
X_train_cp = X_train.copy()

In [18]:
X_test_cp = X_test.copy()

In [22]:
os_path = os.getcwd()
os_path_abs = '/'.join(os_path.split('/')[:-1])
model_path = os.path.join(os_path_abs,'WorkforceSentimentMonitoring/joblib_files')
model_path

'/Users/yunxizzy/code/theresa-sporn/WorkforceSentimentMonitoring/joblib_files'

In [40]:
model_nb_carrer = joblib.load(os.path.join(model_path, 'career-opportunities_nb.joblib'))
model_nb_carrer

Pipeline(steps=[('vectorizer',
                 ColumnTransformer(transformers=[('vectorizer',
                                                  TfidfVectorizer(),
                                                  'review')])),
                ('classifier', MultinomialNB())])

In [23]:
fea_pipe = joblib.load(os.path.join(model_path, 'final_pipe.joblib'))
fea_pipe

Pipeline(steps=[('preprocessor', Preprocessor()),
                ('emo_extractor', EmotionScoresExtractor()),
                ('scaler', CustomMinMaxScaler()),
                ('engineer', FeatureEngineer())])

In [24]:
X_train_fea = fea_pipe.fit_transform(X_train_cp)
X_train_fea

100%|██████████| 36669/36669 [00:51<00:00, 708.49it/s]


,review,length,anger_score,anticipation_score,disgust_score,fear_score,joy_score,sadness_score,surprise_score,trust_score,subjectivity_review,polarity_review
0,great place to work manageable work life balan...,0.011217,0.000000,0.108607,0.000000,0.000000,0.150005,0.000000,0.023050,0.049382,0.650000,0.360000
1,great company work with great mind and have de...,0.006471,0.000000,0.221379,0.048148,0.000000,0.050159,0.000000,0.000000,0.083398,0.666667,0.533333
2,great company work life balance pay benefit co...,0.003883,0.062430,0.391334,0.000000,0.000000,0.166675,0.000000,0.000000,0.256213,0.575000,0.250000
3,great benefit lack morale and fails to provide...,0.057377,0.074583,0.093159,0.137987,0.172008,0.238163,0.140928,0.007622,0.307721,0.531815,0.133586
4,order picker the pay is great and the environm...,0.014668,0.008532,0.356645,0.000000,0.000000,0.359009,0.000000,0.187880,0.118514,0.587500,0.418750
...,...,...,...,...,...,...,...,...,...,...,...,...
36664,sr business analyst work with many talented pe...,0.008197,0.048441,0.094076,0.000000,0.000000,0.000000,0.000000,0.000000,0.121754,0.566667,0.466667
36665,great company to work for really good benefit ...,0.013374,0.090490,0.029486,0.000000,0.000000,0.039156,0.000000,0.016860,0.257766,0.420408,0.309184
36666,star talented co worker unlimited vacation lot...,0.010785,0.000000,0.634980,0.000000,0.000000,0.347627,0.000000,0.000000,0.295705,0.625000,0.325000
36667,senior aha everything is just peachy bruh none...,0.002588,0.000000,0.000000,0.000000,0.000000,0.131263,0.000000,0.000000,0.110916,0.666667,0.333333


# Rerun GridSearch of clf

In [44]:
feature_pipe = joblib.load(os.path.join(model_path, 'feature_pipeline.joblib'))
feature_pipe

Pipeline(steps=[('first_pipe',
                 Pipeline(steps=[('preprocessor', Preprocessor()),
                                 ('emo_extractor', EmotionScoresExtractor()),
                                 ('scaler', CustomMinMaxScaler()),
                                 ('engineer', FeatureEngineer())])),
                ('mnb', MultiNBFeaturesExtractor())])

In [46]:
X_train_cp = X_train.copy()

In [47]:
X_train_final = feature_pipe.fit_transform(X_train_cp)

100%|██████████| 36669/36669 [00:40<00:00, 895.92it/s]


NotFittedError: This ColumnTransformer instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [ ]:
y_train